In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests
from bs4 import BeautifulSoup

In [2]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page = requests.get(wikipedia_link)
page= raw_wikipedia_page.text
soup = BeautifulSoup(page, 'lxml')


table = soup.find("table")
df  = pd.DataFrame( columns = ['Postal Code', 'Borough', 'Neighborhood'])
data = {}

i = 0
for row in table.findAll("tr"):
    cells = row.findAll("td")
    ls= []
    for cell in cells:
        ls.append(cell.find(text=True))
        
    data[i]= ls
    i = i+1

for x in range (1, len(data)):
    if data[x][1] == 'Not assigned':
        continue
    else:  
        if data[x][2].strip() == 'Not assigned':
            data[x][2] = data[x][1]
            
        
        df = df.append(
        {
          'Postal Code': data[x][0],
          'Borough': data[x][1],
          'Neighborhood': data[x][2]
        }, ignore_index=True)  
       

df = df.reset_index(drop=True)
#print (df.head(20))
modified_data = {}
for index, row in df.iterrows():
    if row['Postal Code'] in modified_data.keys():
        modified_data[row['Postal Code']][2] = modified_data[row['Postal Code']][2].replace('\n','') + ',' + row['Neighborhood'].replace('\n','')
    else:    
        modified_data[row['Postal Code']] = [row['Postal Code'], row['Borough'].replace('\n',''), row['Neighborhood'].replace('\n','')]

#print ( modified_data)
dfObj = pd.DataFrame(modified_data).T 
dfObj = dfObj.reset_index(drop=True)
dfObj.columns =  ['Postal Code', 'Borough', 'Neighborhood']
print(dfObj.head(10))

  Postal Code           Borough                     Neighborhood
0         M3A        North York                        Parkwoods
1         M4A        North York                 Victoria Village
2         M5A  Downtown Toronto         Harbourfront,Regent Park
3         M6A        North York  Lawrence Heights,Lawrence Manor
4         M7A      Queen's Park                     Queen's Park
5         M9A         Etobicoke                 Islington Avenue
6         M1B       Scarborough                    Rouge,Malvern
7         M3B        North York                  Don Mills North
8         M4B         East York   Woodbine Gardens,Parkview Hill
9         M5B  Downtown Toronto          Ryerson,Garden District


In [6]:
url = 'http://cocl.us/Geospatial_data'
data = pd.read_csv(url)
df_geocode  = pd.DataFrame( columns = ['Postal Code', 'Borough', 'Neighborhood', 'Latitude',  'Longitude'])
df_geocode = pd.merge(dfObj, data, on='Postal Code')
print(df_geocode.head(2))

  Postal Code     Borough      Neighborhood   Latitude  Longitude
0         M3A  North York         Parkwoods  43.753259 -79.329656
1         M4A  North York  Victoria Village  43.725882 -79.315572


In [4]:
dfObj.shape

(103, 3)